## Entity Extraction in Comparative Analysis

Entity extraction is fundamental to comparative analysis as it identifies the core subjects of comparison within subjective comparative questions. This involves pinpointing and categorizing the entities (such as products, services, or features) that are being compared. Effective entity extraction ensures that each comparative relation is anchored to clear, identifiable elements, making the subsequent analysis more structured and interpretable. This process is essential for constructing a reliable foundation for further comparative assessments and is integral to developing accurate and context-aware NLP solutions.


## Setting Up the Environment

Before diving into our classification project, we must ensure our Python environment has all the necessary libraries. We'll use the `transformers` library for accessing pre-trained models like RoBERTa, which is essential for our NLP tasks. PyTorch, along with its `torchvision` and `torchaudio` libraries, provides a comprehensive ecosystem for building and training deep learning models. Additionally, scikit-learn offers valuable tools for data preprocessing, model evaluation, and implementing cross-validation strategies.

**Required Libraries:**

* **transformers:** Provides pre-trained models for various NLP tasks.
* **torch, torchvision, torchaudio:** PyTorch ecosystem for deep learning operations.
* **scikit-learn:** Offers machine learning tools for data manipulation and model evaluation.

**Installation:**

You can install these libraries using `pip` within a code cell:

In [2]:
#!pip install --upgrade transformers
!pip install torch torchvision torchaudio
!pip install transformers
!pip install scikit-learn

## Data Acquisition

Before diving into the intricacies of our Element Extraction model, the first crucial step is to acquire the relevant dataset. In the following code snippet, we utilize the `requests` library to download a pre-compiled dataset from a specified URL. Upon successful download, we proceed to unzip


In [3]:
import requests
import zipfile
import os

data_url = "https://github.com/mahsamb/SCRQD/raw/main/Dataset.zip"
zip_filename = "Dataset.zip"

# Downloading using requests
response = requests.get(data_url)

# Check if the request was successful (status_code 200)
if response.status_code == 200:
    with open(zip_filename, "wb") as f:
        f.write(response.content)
else:
    print(f"Failed to retrieve the data: {response.status_code}: {response.text}")
    # Add additional error handling here

# Unzipping the dataset
try:
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall("data")
    print("Files extracted:")
    print(os.listdir("data"))
except zipfile.BadZipFile:
    print("Error: The file doesn’t appear to be a valid zip file")


Files extracted:
['Questions.pkl', 'Elements.pkl', 'Relations.pkl', 'EntityRoles.pkl', 'ComparativePreferences.pkl']


## Preparing the Environment and Loading Data

To ensure our Element Extraction model functions effectively, we start by setting up our environment with essential libraries such as `numpy` and `pandas` for data manipulation, and `re` for regular expressions, which are critical for processing text data. Additionally, we use `pickle` for loading our


In [4]:
import numpy as np
import pickle as cPickle
import pickle
import re
import pandas as pd
from IPython.display import display, HTML
import random



with open(r"/kaggle/working/data/Questions.pkl", "rb") as input_file:
    QuestionDict = pickle.load(input_file)
    input_file.close()


with open(r"/kaggle/working/data/Elements.pkl", "rb") as input_file:
    Product_Aspect_Contraint_dict = pickle.load(input_file)
    input_file.close()

## Initial Exploration of Loaded Datasets

With our datasets loaded into Python dictionaries, it's essential to begin with an initial exploration to understand the structure and type of data we'll be working with. In the code snippets provided, we iterate through both `Product_Aspect_Contraint_dict` and `QuestionDict`, printing the first key-value pair of each to get a glimpse into the data. This preliminary step is crucial for ensuring the integrity of our data and to familiarize ourselves with the dataset's format, which will inform our strategy for the Element Extraction process.


In [5]:
for key, value in Product_Aspect_Contraint_dict.items():
  print(key)
  print(value)
  break

for key, value in QuestionDict.items():
  print(key)
  print(value)
  break

1
[['O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P B-P O-P'], ['O-A O-A O-A O-A O-A O-A O-A B-A I-A I-A I-A O-A O-A O-A B-A I-A O-A B-A O-A O-A O-A O-A O-A'], ['O-C O-C O-C O-C B-C B-C I-C I-C I-C I-C I-C O-C O-C O-C O-C O-C O-C O-C O-C B-C I-C I-C O-C']]
1
What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?


## Constructing the Data Structure for Model Input

After our initial exploration, the next step involves structuring our data into a format suitable for our Element Extraction model. The provided code snippet accomplishes this by iterating over the `QuestionDict` and corresponding `Product_Aspect_Contraint_dict` to create a comprehensive list of dictionaries, each containing a question and its associated entity, aspect, and constraint labels. This structured approach facilitates the easy manipulation and analysis of our dataset, preparing it for the model training phase. By aligning our questions with their respective labels in a unified data structure, we ensure a smooth transition into the model's training and evaluation stages.


In [6]:
data = []

for key, value in QuestionDict.items():
    question_text = value  # The question text from QuestionDict
    label_info = Product_Aspect_Contraint_dict[key]  # The corresponding labels from Product_Aspect_Constraint_dict

    # Extract label lists directly from label_info without trying to split them
    entity_labels, aspect_labels, constraint_labels = label_info

    # Since label_info items are already lists, we can directly use them
    # Adjust the structure as needed based on your exact format

    data_entry = {
        "text": question_text,
        "entity_labels": entity_labels,  # No need to wrap in another list or call split
        "aspect_labels": aspect_labels,
        "constraint_labels": constraint_labels
    }

    data.append(data_entry)

## Previewing Structured Data Entries

To verify the integrity and structure of our newly constructed data entries, we print the first item from our prepared data list. This step is crucial for ensuring that our data is correctly formatted and contains all necessary information for the Element Extraction tasks. By examining this sample entry, we can confirm the successful preparation of our data, setting the stage for the subsequent model training and evaluation processes.


In [7]:
for item in data:
    print(item)
    #print(item.type)
    break

{'text': 'What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?', 'entity_labels': ['O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P O-P B-P O-P'], 'aspect_labels': ['O-A O-A O-A O-A O-A O-A O-A B-A I-A I-A I-A O-A O-A O-A B-A I-A O-A B-A O-A O-A O-A O-A O-A'], 'constraint_labels': ['O-C O-C O-C O-C B-C B-C I-C I-C I-C I-C I-C O-C O-C O-C O-C O-C O-C O-C O-C B-C I-C I-C O-C']}


## Refining Label Formats for Analysis

Before we proceed with training our Element Extraction model, it's imperative to refine the format of our labels to ensure compatibility with our processing algorithms. This code snippet updates each data entry by splitting the label strings into lists of individual labels. This step transforms our previously structured label information into a more granular and model-friendly format, enabling precise tagging and classification during the training phase. By adjusting the label format, we enhance the model's ability to learn from and accurately predict the elements within our dataset.



In [8]:
# Iterate through each dictionary in the list
for item in data:
    # Splitting the labels based on space ' ' and updating the values
    item['entity_labels'] = item['entity_labels'][0].split()
    item['aspect_labels'] = item['aspect_labels'][0].split()
    item['constraint_labels'] = item['constraint_labels'][0].split()

#print(data)
for item in data:
    print(item)
    break

{'text': 'What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?', 'entity_labels': ['O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'O-P', 'B-P', 'O-P'], 'aspect_labels': ['O-A', 'O-A', 'O-A', 'O-A', 'O-A', 'O-A', 'O-A', 'B-A', 'I-A', 'I-A', 'I-A', 'O-A', 'O-A', 'O-A', 'B-A', 'I-A', 'O-A', 'B-A', 'O-A', 'O-A', 'O-A', 'O-A', 'O-A'], 'constraint_labels': ['O-C', 'O-C', 'O-C', 'O-C', 'B-C', 'B-C', 'I-C', 'I-C', 'I-C', 'I-C', 'I-C', 'O-C', 'O-C', 'O-C', 'O-C', 'O-C', 'O-C', 'O-C', 'O-C', 'B-C', 'I-C', 'I-C', 'O-C']}


## Applying Label Encoding for Machine Learning

The final preprocessing step involves converting our textual labels into numerical representations, a process known as label encoding. This transformation is crucial for machine learning models, which require numerical input. Using a predefined mapping dictionary, we replace the textual labels in our dataset with their corresponding numerical codes. This encoding not only simplifies the model's input but also ensures that the training process is optimized for efficiency and accuracy. By examining a sample entry after this transformation, we can confirm the readiness of our data for the upcoming machine learning tasks.

In [9]:
# Mapping dictionary
label_map = {"O-P": 0, "B-P": 1, "I-P": 2, "O-A": 0, "B-A": 1, "I-A": 2, "O-C": 0, "B-C": 1, "I-C": 2}

# Iterate through each dictionary in the list
for item in data:
    # Replace entity_labels
    item['entity_labels'] = [label_map[label] for label in item['entity_labels']]
    # Replace aspect_labels
    item['aspect_labels'] = [label_map[label] for label in item['aspect_labels']]
    # Replace constraint_labels
    item['constraint_labels'] = [label_map[label] for label in item['constraint_labels']]

    #print(data)
for item in data:
    print(item)
    break

{'text': 'What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?', 'entity_labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'aspect_labels': [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0], 'constraint_labels': [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]}


## Displaying Processed Data and Labels

To confirm that our data is correctly processed and ready for the next steps, we display the text, entity labels, aspect labels, and constraint labels of the first entry in our prepared dataset. This visualization allows us to ensure that the textual data aligns with its corresponding numerical labels, indicating successful label encoding and data structuring. It's a vital checkpoint to verify the data's readiness for model training and analysis.

In [10]:
print(data[0]['text'])
print(data[0]['entity_labels'])
print(data[0]['aspect_labels'])
print(data[0]['constraint_labels'])

What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]


## Organizing Data for Easy Access

As we move forward, organizing our data in a manner that facilitates easy access and manipulation becomes crucial. By categorizing the processed data into separate dictionaries for entities, aspects, and constraints, we enhance our ability to swiftly retrieve and analyze specific segments of our dataset. This organization not only aids in data management but also in performing focused analyses on different elements of the text. Displaying the structured data for the first entry further solidifies our understanding of the dataset's organization and prepares us for detailed element extraction tasks.

In [11]:
# Assuming "data" is a list of dictionaries, each structured like the example you provided

# Initialize three empty dictionaries
entity_dict, aspect_dict, constraint_dict = {}, {}, {}

# Iterate through each item in the data list
for item in data:
    # Extract and assign the relevant information to each dictionary
    entity_dict[item['text']] = item['entity_labels']
    aspect_dict[item['text']] = item['aspect_labels']
    constraint_dict[item['text']] = item['constraint_labels']

# Example to show the result for the first item in the list
print("Text:", data[0]['text'])
print("Entity Labels:", entity_dict[data[0]['text']])
print("Aspect Labels:", aspect_dict[data[0]['text']])
print("Constraint Labels:", constraint_dict[data[0]['text']])


Text: What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?
Entity Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Aspect Labels: [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0]
Constraint Labels: [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]


## Extracting and Demonstrating Key-Value Pair Relationships

The function `extract_values` is designed to map keys from one list to values in another, based on a reference dictionary. This is particularly useful in scenarios where we need to align different elements of our dataset, such as texts and their corresponding labels, for detailed analysis or processing. Following this, we showcase the utility of this function by preparing to align texts with their labels for entities, aspects, and constraints separately. By extracting and printing the first elements of these lists, we prepare to demonstrate how texts and labels can be effectively paired, providing a clear foundation for any subsequent operations that require a direct association between textual data and their annotated labels.

In [13]:
def extract_values(dict1, list1, list2):
    result = {}
    for key, value in zip(list1, list2):
        result[key] = dict1.get(value, None)
    return result


entity_text_ = list(entity_dict.keys())
entity_labels_ = list(entity_dict.values())

print("list1:", entity_text_[0])
print("list2:", entity_labels_[0])


aspect_text_ = list(aspect_dict.keys())
aspect_labels_ = list(aspect_dict.values())

print("list1:", aspect_text_[0])
print("list2:", aspect_labels_[0])


constraint_text_ = list(constraint_dict.keys())
constraint_labels_ = list(constraint_dict.values())

print("list1:", constraint_text_[0])
print("list2:", constraint_labels_[0])

list1: What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?
list2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
list1: What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?
list2: [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0]
list1: What are the best smartphones with a built in stylus feature with a good quality display and RAM , other than Samsung ?
list2: [0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]


## Preparing Data for RoBERTa Token Classification

To fine-tune the RoBERTa model for token classification, we first import necessary modules and initialize the RoBERTa tokenizer and model with the appropriate number of labels for our classification task. We then proceed to preprocess our entity texts and labels for model input. This involves tokenizing the texts, creating attention masks for handling padding, and adjusting label lengths to match the tokenized inputs. Padding labels with -100 ensures that these positions are ignored during the loss calculation, aligning with the model's requirements. By converting these lists into tensors, we prepare our dataset for training with RoBERTa, setting the stage for effective learning and classification of entities, aspects, and constraints within our text data.

In [14]:
import torch
from transformers import RobertaTokenizer, RobertaForTokenClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForTokenClassification.from_pretrained('roberta-base', num_labels=3)


max1=100
max_length = max1  # Adjust as needed

input_ids = []
attention_masks = []
entity_labels = []

texts=[]
labels=[]

texts = entity_text_
labels = entity_labels_

for text, label in zip(texts, labels):
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=max_length,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                    )

    # Create a mask for the labels to ignore padding in the loss computation
    label_mask = [1] * len(label) + [0] * (max_length - len(label))
    label = label + [-100] * (max_length - len(label))  # Padding labels with -100, which is ignored by the loss function

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
    entity_labels.append(label)

entity_input_ids = torch.cat(input_ids, dim=0)
entity_attention_masks = torch.cat(attention_masks, dim=0)
entity_labels = torch.tensor(entity_labels)

# Proceed with dataset creation, DataLoader setup, and cross-validation as before

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Implementing K-Fold Cross-Validation for Model Evaluation
To rigorously evaluate our RoBERTa model's performance on the entity classification task, we employ a K-Fold cross-validation strategy. This method divides our dataset into five distinct folds, systematically using each fold as a validation set while training on the remaining data. Throughout the training process, we adjust the model's parameters using the Adam optimizer and track metrics such as loss and accuracy for both training and validation phases. By calculating precision, recall, and F1-score after each fold, we gain insights into the model's generalization ability and its performance across different subsets of the data. This comprehensive evaluation approach not only helps in fine-tuning the model's parameters but also ensures that we develop a robust and reliable classification system.

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
from transformers import RobertaModel
#from transformers import AdamW
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
from tqdm.auto import tqdm
from torch.optim import Adam,AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data.dataset import random_split



num_labels = 3
num_epochs = 5

class SingleTaskRoberta(nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        logits = self.classifier(sequence_output)
        return logits
    


class TokenClassificationDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx],
        }

# Function to compute accuracy
def calculate_accuracy(logits, labels):
    preds = torch.argmax(logits, dim=-1)
    mask = labels != -100
    preds = preds[mask]
    labels = labels[mask]
    accuracy = (preds == labels).float().mean()
    return accuracy.item()


# Initialize your single_task_dataset here with your real data
single_task_dataset = TokenClassificationDataset(entity_input_ids, entity_attention_masks, entity_labels)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SingleTaskRoberta(num_labels=3).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_fold_results = []

for fold, (train_ids, test_ids) in enumerate(kf.split(single_task_dataset)):
    print(f"\nStarting Fold {fold + 1}")

    # Splitting the dataset into training and validation subsets
    train_subset = Subset(single_task_dataset, train_ids)
    val_size = int(0.1 * len(train_subset))  # For example, 10% for validation
    train_size = len(train_subset) - val_size
    train_dataset, val_dataset = random_split(train_subset, [train_size, val_size])
    
    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=8)  # Validation DataLoader
    test_dataloader = DataLoader(Subset(single_task_dataset, test_ids), batch_size=8)

 
    ############

    model = SingleTaskRoberta(num_labels=3).to(device)

    optimizer = AdamW(model.parameters(), lr=5e-5)



    for epoch in range(num_epochs):  # Number of epochs
        model.train()
        total_loss, total_accuracy = 0, 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)

            loss = criterion(logits.view(-1, num_labels), labels.view(-1))
            loss.backward()
            optimizer.step()
            #scheduler.step()  # Update the learning rate
           

            total_loss += loss.item()
            accuracy = calculate_accuracy(logits, labels)
            total_accuracy += accuracy

        epoch_loss = total_loss / len(train_dataloader)
        epoch_accuracy = total_accuracy / len(train_dataloader)
        print(f"Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.4f}")

        model.eval()
        total_val_loss, total_val_accuracy = 0, 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                logits = model(input_ids, attention_mask)
                val_loss = criterion(logits.view(-1, num_labels), labels.view(-1))

                total_val_loss += val_loss.item()
                val_accuracy = calculate_accuracy(logits, labels)
                total_val_accuracy += val_accuracy

        
        val_accuracy = total_val_accuracy / len(val_dataloader)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # Test phase for the current fold
        test_preds, test_labels = [], []
        model.eval()
        with torch.no_grad():
            for batch in test_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                logits = model(input_ids, attention_mask)
                preds = torch.argmax(logits, dim=-1).detach().cpu().numpy()
                labels = labels.cpu().numpy()

                # Filter out '-100' used for padding in labels
                active = labels != -100
                test_preds.extend(preds[active])
                test_labels.extend(labels[active])

    precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_preds, average='macro', zero_division=0)
    print(f"\nFold {fold + 1} - Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    
    all_fold_results.append({'precision': precision, 'recall': recall, 'f1': f1})
    
# Calculate and print average metrics after all folds
avg_precision = np.mean([fold['precision'] for fold in all_fold_results])
avg_recall = np.mean([fold['recall'] for fold in all_fold_results])
avg_f1 = np.mean([fold['f1'] for fold in all_fold_results])

print(f"\nAverage Across All Folds - Precision: {avg_precision:.4f}, Recall: {avg_recall:.4f}, F1: {avg_f1:.4f}")



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Starting Fold 1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1:   0%|          | 0/115 [00:00<?, ?it/s]

Training Loss: 0.3014, Training Accuracy: 0.8990
Validation Loss: 0.0746, Validation Accuracy: 0.9384


...

Training Loss: 0.1215, Training Accuracy: 0.9618
Validation Loss: 0.2104, Validation Accuracy: 0.9578


Epoch 4:   0%|          | 0/115 [00:00<?, ?it/s]

Training Loss: 0.0874, Training Accuracy: 0.9701
Validation Loss: 0.0233, Validation Accuracy: 0.9798


Epoch 5:   0%|          | 0/115 [00:00<?, ?it/s]

Training Loss: 0.0755, Training Accuracy: 0.9748
Validation Loss: 0.0483, Validation Accuracy: 0.9787

Fold 5 - Test Precision: 0.9610, Recall: 0.9646, F1: 0.9627

Average Across All Folds - Precision: 0.9559, Recall: 0.9649, F1: 0.9603
